Comienzo de la práctica

In [1]:
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd #procesado del csv pasa E/S
import matplotlib as plt #printeo
import random #operaciones matematicas
from metodos import * #importamos metodos para una mayor legibilidad

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import os 
for dirname, _, filenames in os.walk('/home/ubuntu/Escritorio/repos/segundo-cuatri/NLP/NLP-practicas_grupo/proyecto-final'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/ubuntu/Escritorio/repos/segundo-cuatri/NLP/NLP-practicas_grupo/proyecto-final/Temas.txt
/home/ubuntu/Escritorio/repos/segundo-cuatri/NLP/NLP-practicas_grupo/proyecto-final/notebook_final.ipynb
/home/ubuntu/Escritorio/repos/segundo-cuatri/NLP/NLP-practicas_grupo/proyecto-final/metodos.py
/home/ubuntu/Escritorio/repos/segundo-cuatri/NLP/NLP-practicas_grupo/proyecto-final/archive/AI_Human.csv
/home/ubuntu/Escritorio/repos/segundo-cuatri/NLP/NLP-practicas_grupo/proyecto-final/.ipynb_checkpoints/notebook_final-checkpoint.ipynb
/home/ubuntu/Escritorio/repos/segundo-cuatri/NLP/NLP-practicas_grupo/proyecto-final/__pycache__/metodos.cpython-310.pyc


In [3]:
datos = pd.read_csv('/home/ubuntu/Escritorio/repos/segundo-cuatri/NLP/NLP-practicas_grupo/proyecto-final/archive/AI_Human.csv')
datos.head()

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [4]:
datos_totales = (datos['generated']).count()
datos_humanos = (datos['generated'] == 0.0).sum()
print("Datos totales: ", datos_totales)
print("Humanos: ", datos_humanos)
print("IA: ", datos_totales - datos_humanos)

Datos totales:  487235
Humanos:  305797
IA:  181438


In [5]:
token = nltk.word_tokenize(datos['text'][0])
nltk.pos_tag(token)

[('Cars', 'NNS'),
 ('.', '.'),
 ('Cars', 'NNS'),
 ('have', 'VBP'),
 ('been', 'VBN'),
 ('around', 'IN'),
 ('since', 'IN'),
 ('they', 'PRP'),
 ('became', 'VBD'),
 ('famous', 'JJ'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('1900s', 'CD'),
 (',', ','),
 ('when', 'WRB'),
 ('Henry', 'NNP'),
 ('Ford', 'NNP'),
 ('created', 'VBD'),
 ('and', 'CC'),
 ('built', 'VBD'),
 ('the', 'DT'),
 ('first', 'JJ'),
 ('ModelT', 'NNP'),
 ('.', '.'),
 ('Cars', 'NNS'),
 ('have', 'VBP'),
 ('played', 'VBN'),
 ('a', 'DT'),
 ('major', 'JJ'),
 ('role', 'NN'),
 ('in', 'IN'),
 ('our', 'PRP$'),
 ('every', 'DT'),
 ('day', 'NN'),
 ('lives', 'VBZ'),
 ('since', 'IN'),
 ('then', 'RB'),
 ('.', '.'),
 ('But', 'CC'),
 ('now', 'RB'),
 (',', ','),
 ('people', 'NNS'),
 ('are', 'VBP'),
 ('starting', 'VBG'),
 ('to', 'TO'),
 ('question', 'VB'),
 ('if', 'IN'),
 ('limiting', 'JJ'),
 ('car', 'NN'),
 ('usage', 'NN'),
 ('would', 'MD'),
 ('be', 'VB'),
 ('a', 'DT'),
 ('good', 'JJ'),
 ('thing', 'NN'),
 ('.', '.'),
 ('To', 'TO'),
 ('me', 'PRP'),
 (',',

In [6]:
tokens_ia = []
tokens_human = []
i = (len(datos['text'])//20)
while i > 0:
    valor = random.randint(0, len(datos['text']))
    token = nltk.word_tokenize(datos['text'][valor])
    if datos['generated'][i] == 1.0:
        tokens_ia.append(token)
    else:
        tokens_human.append(token)
    i -= 1


KeyError: 487235

Let's find the lexical richness of the AI texts

In [ ]:
medium_richness_ia = 0
num = 0
for i in range(0, len(tokens_ia)):
    medium_richness_ia += lexical_richness(tokens_ia[i])
    num += 1
medium_richness_ia = medium_richness_ia/num
medium_richness_ia

0.438207868213972

Let's find the lexical richness of the Human texts

In [ ]:
medium_richness_human = 0
num = 0
for i in range(0, len(tokens_human)):
    medium_richness_human += lexical_richness(tokens_human[i])
    num += 1
medium_richness_human = medium_richness_human/num
medium_richness_human

0.4375080482835001

We're going to chech the performance and we're gonna display the accuracy of the tagged words

In [ ]:
tagged_sents = tokens_ia

def pos_tagged_words(tagged_sents):
    tagged_words = []
    for sent in tagged_sents:
        for word in sent:
            tagged_words.append(word)
    return tagged_words

tagged_words = pos_tagged_words(tagged_sents)
tagged_words

['Cars',
 'are',
 'a',
 'major',
 'source',
 'of',
 'pollution',
 ',',
 'noise',
 ',',
 'and',
 'congestion',
 'in',
 'urban',
 'areas',
 '.',
 'As',
 'a',
 'consequence',
 ',',
 'some',
 'cities',
 'have',
 'decided',
 'to',
 'go',
 'car-free',
 ',',
 'or',
 'at',
 'least',
 'significantly',
 'reduce',
 'car',
 'usage',
 '.',
 'This',
 'shift',
 'can',
 'have',
 'numerous',
 'benefits',
 'for',
 'the',
 'environment',
 ',',
 'public',
 'health',
 ',',
 'and',
 'quality',
 'of',
 'life',
 '.',
 '1',
 '.',
 'Improved',
 'air',
 'quality',
 ':',
 'With',
 'fewer',
 'cars',
 'on',
 'the',
 'roads',
 ',',
 'cities',
 'can',
 'experience',
 'a',
 'significant',
 'reduction',
 'in',
 'pollution',
 'levels',
 '.',
 'This',
 'leads',
 'to',
 'cleaner',
 'air',
 ',',
 'less',
 'asthma',
 ',',
 'and',
 'other',
 'respiratory',
 'issues',
 '.',
 'In',
 'car-free',
 'cities',
 ',',
 'people',
 'tend',
 'to',
 'spend',
 'more',
 'time',
 'outdoors',
 ',',
 'as',
 'they',
 'do',
 "n't",
 'need',
 't

In [ ]:
fd = nltk.FreqDist(tagged_sents)
cfd = nltk.ConditionalFreqDist(tagged_sents)
most_freq_words = fd.most_common(10)
most_freq_words
#FIXME: No tengo ni puta idea de que estoy haciendo

TypeError: unhashable type: 'list'